In [1]:
## Importing the relevant packages:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from statistics import mean
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import nltk

In [2]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Wajeeha\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [3]:
data_df = pd.read_csv("decision_maker_survey_data.csv")
print(len(data_df))
data_df.head()

442


,ResponseId,UserLanguage,consent,employment_status,role_type,misinfo_intro,misinfo_attitudes_general,misinfo_attitudes_companies,misinfo_attitudes_platforms,attitudes_recommend,...,verified_DM,final_exec,final_marketing,final_manager,final_other,appears_on_misinfo,appears_in_MP_data,correct_company_beliefs,follow_up_ad_check_surprise,surprised_by_ad_check
0,R_TwtImxSVd1qRcIh,EN,I AGREE to participate in this study and confi...,Full-time employee,Business and finance specialist (business spe...,I have carefully read the above information.,Strongly agree,Strongly agree,Strongly agree,Strongly agree,...,0,0,0,0,1,0,0,0,NaN,None
1,R_3PRIChFajTvXyKA,EN,I AGREE to participate in this study and confi...,Self-employed or small business owner,"Top executive (e.g. chief executives, general...",I have carefully read the above information.,Agree,Somewhat agree,Agree,Somewhat agree,...,1,1,0,0,0,0,0,1,NaN,None
2,R_2dQPRTL3YmpF5OJ,EN,I AGREE to participate in this study and confi...,Self-employed or small business owner,Operations manager (e.g. managers of administ...,I have carefully read the above information.,Strongly agree,Agree,Strongly agree,Agree,...,0,0,0,1,0,1,1,0,NaN,None
3,R_1LNn1eL3N1gVIrp,EN,I AGREE to participate in this study and confi...,Full-time employee,"Top executive (e.g. chief executives, general...",I have carefully read the above information.,Agree,Agree,Agree,Agree,...,1,1,0,0,0,0,0,1,Unsurprising,0
4,R_11XzapZQcNaDhm9,EN,I AGREE to participate in this study and confi...,Full-time employee,"Manager of advertising, marketing, promotions,...",I have carefully read the above information.,Strongly agree,Strongly agree,Agree,Agree,...,0,0,1,1,0,0,0,1,NaN,None


In [4]:
df = data_df[['treated', 'purpose']]
print(len(df))

df['label'] = df['treated']
df['label'].value_counts()

442


C:\Users\Wajeeha\AppData\Local\Temp\ipykernel_11884\4163287486.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['treated']


0    226
1    216
Name: label, dtype: int64

In [5]:
df.head()

,treated,purpose,label
0,0,NaN,0
1,1,To obtain information on how big of a problem ...,1
2,0,Educate on the misinformation out there for bu...,0
3,1,create a tool for safer advertising,1
4,0,To understand peoples beliefs and preferences ...,0


In [6]:
print(len(df))
df['text'] = df['purpose']
df = df[df['text'].notna()]
df['text'] = df['text'].astype(str)
df['text'] = [entry.lower() for entry in df['text']]
df['text']= [word_tokenize(entry) for entry in df['text']]
print(len(df))
df = df.reset_index()
df.head()

C:\Users\Wajeeha\AppData\Local\Temp\ipykernel_11884\2673384535.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['purpose']


442
356


,index,treated,purpose,label,text
0,1,1,To obtain information on how big of a problem ...,1,"[to, obtain, information, on, how, big, of, a,..."
1,2,0,Educate on the misinformation out there for bu...,0,"[educate, on, the, misinformation, out, there,..."
2,3,1,create a tool for safer advertising,1,"[create, a, tool, for, safer, advertising]"
3,4,0,To understand peoples beliefs and preferences ...,0,"[to, understand, peoples, beliefs, and, prefer..."
4,5,1,whether changing the benefits listed in the em...,1,"[whether, changing, the, benefits, listed, in,..."


In [7]:
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

for index, entry in enumerate(df['text']):
    Final_words = []
    word_Lemmatized = WordNetLemmatizer()    
    for word, tag in pos_tag(entry):
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word, tag_map[tag[0]])
            Final_words.append(word_Final)
    df.loc[index, 'text_final'] = str(Final_words)
    
print(len(df))
df['text_final'] = df['text_final'].astype(str)
df.head()


KeyboardInterrupt



In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(df['text_final'], df['label'], test_size = 0.25)

Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

print(len(Train_X), len(Test_X), len(Train_X) + len(Test_X))
print(len(Train_Y), len(Test_Y), len(Train_Y) + len(Test_Y))

Tfidf_vect = TfidfVectorizer(max_features = 5000)
Tfidf_vect.fit(df['text_final'])

Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

Test_X_Tfidf 

In [ ]:
# Classifier - Algorithm - SVM
# Fit the training dataset on the classifier
SVM = svm.SVC(C = 1.0, kernel = 'linear', degree = 3, gamma = 'auto')
SVM.fit(Train_X_Tfidf, Train_Y)

# Predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, Test_Y) * 100)

print(len(predictions_SVM))
#predictions_SVM

In [ ]:
test_df = pd.DataFrame(Test_X)
test_df = test_df.reset_index()
test_df

test_df['predicted_label'] = predictions_SVM
test_df['actual_label'] = Test_Y
print(test_df['actual_label'].sum(), test_df['predicted_label'].sum())
test_df

In [ ]:
def get_categories(df):
    if ((df['actual_label'] == 1) & (df['predicted_label'] == 1)):
        return "True Treated & Predicted Treated"
    
    elif ((df['actual_label'] == 1) & (df['predicted_label'] == 0)):
        return "True Treated & Predicted Control"
    
    elif ((df['actual_label'] == 0) & (df['predicted_label'] == 1)):
        return "True Control & Predicted Treated"
    
    elif ((df['actual_label'] == 0) & (df['predicted_label'] == 0)):
        return "True Control & Predicted Control"
    
    else:
        return 0
    
test_df['categories'] = test_df.apply(get_categories, axis = 1)
test_df['categories'].value_counts()